In [30]:
import cv2
import numpy as np

In [31]:
infile = '/home/nlbutts/Videos/out.mp4'
trim_front = 5
time_back = 1

In [45]:
cap = cv2.VideoCapture(infile)
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
print(f'{w}x{h} total frames: {n_frames} fps: {fps}')
print(cap.isOpened())

1280x720 total frames: 450 fps: 29.97002997002997
True


In [33]:
_, prev = cap.read()
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

In [20]:
def movingAverage(curve, radius):
  window_size = 2 * radius + 1
  # Define the filter
  f = np.ones(window_size)/window_size
  # Add padding to the boundaries
  curve_pad = np.pad(curve, (radius, radius), 'edge')
  # Apply convolution
  curve_smoothed = np.convolve(curve_pad, f, mode='same')
  # Remove padding
  curve_smoothed = curve_smoothed[radius:-radius]
  # return smoothed curve
  return curve_smoothed


In [21]:
def smooth(trajectory, SMOOTHING_RADIUS=5):
  smoothed_trajectory = np.copy(trajectory)
  # Filter the x, y and angle curves
  for i in range(3):
    smoothed_trajectory[:,i] = movingAverage(trajectory[:,i], radius=SMOOTHING_RADIUS)

  return smoothed_trajectory


In [14]:
# Pre-define transformation-store array
transforms = np.zeros((n_frames-1, 3), np.float32)

for i in range(n_frames-2):
  # Detect feature points in previous frame
  prev_pts = cv2.goodFeaturesToTrack(prev_gray,
                                     maxCorners=200,
                                     qualityLevel=0.01,
                                     minDistance=30,
                                     blockSize=3)

  # Read next frame
  success, curr = cap.read()
  if not success:
    break

  # Convert to grayscale
  curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)

  # Calculate optical flow (i.e. track feature points)
  curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None)

  # Sanity check
  assert prev_pts.shape == curr_pts.shape

  # Filter only valid points
  idx = np.where(status==1)[0]
  prev_pts = prev_pts[idx]
  curr_pts = curr_pts[idx]

  #Find transformation matrix
  #m = cv2.estimateRigidTransform(prev_pts, curr_pts, fullAffine=False) #will only work with OpenCV-3 or less
  m, inliers = cv2.estimateAffine2D(prev_pts, curr_pts)

  # Extract traslation
  dx = m[0,2]
  dy = m[1,2]

  # Extract rotation angle
  da = np.arctan2(m[1,0], m[0,0])

  # Store transformation
  transforms[i] = [dx,dy,da]

  # Move to next frame
  prev_gray = curr_gray

  print("Frame: " + str(i) +  "/" + str(n_frames) + " -  Tracked points : " + str(len(prev_pts)))

Frame: 0/450 -  Tracked points : 196
Frame: 1/450 -  Tracked points : 200
Frame: 2/450 -  Tracked points : 198
Frame: 3/450 -  Tracked points : 196
Frame: 4/450 -  Tracked points : 198
Frame: 5/450 -  Tracked points : 200
Frame: 6/450 -  Tracked points : 200
Frame: 7/450 -  Tracked points : 198
Frame: 8/450 -  Tracked points : 194
Frame: 9/450 -  Tracked points : 198
Frame: 10/450 -  Tracked points : 198
Frame: 11/450 -  Tracked points : 198
Frame: 12/450 -  Tracked points : 199
Frame: 13/450 -  Tracked points : 198
Frame: 14/450 -  Tracked points : 199
Frame: 15/450 -  Tracked points : 199
Frame: 16/450 -  Tracked points : 200
Frame: 17/450 -  Tracked points : 199
Frame: 18/450 -  Tracked points : 197
Frame: 19/450 -  Tracked points : 196
Frame: 20/450 -  Tracked points : 199
Frame: 21/450 -  Tracked points : 193
Frame: 22/450 -  Tracked points : 198
Frame: 23/450 -  Tracked points : 198
Frame: 24/450 -  Tracked points : 199
Frame: 25/450 -  Tracked points : 200
Frame: 26/450 -  Track

In [46]:
# Compute trajectory using cumulative sum of transformations
trajectory = np.cumsum(transforms, axis=0)
smoothed_trajectory = smooth(trajectory, 50)

In [47]:
# Calculate difference in smoothed_trajectory and trajectory
difference = smoothed_trajectory - trajectory

# Calculate newer transformation array
transforms_smooth = transforms + difference


In [48]:
def fixBorder(frame):
  s = frame.shape
  # Scale the image 4% without moving the center
  T = cv2.getRotationMatrix2D((s[1]/2, s[0]/2), 0, 1.04)
  frame = cv2.warpAffine(frame, T, (s[1], s[0]))
  return frame


In [49]:
# Reset stream to first frame
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

fourcc = cv2.VideoWriter_fourcc(*'yuv4')
out = cv2.VideoWriter('video_out.mkv', fourcc, fps, (w,h))

# Write n_frames-1 transformed frames
for i in range(n_frames-2):
  # Read next frame
  success, frame = cap.read()
  if not success:
    break

  # Extract transformations from the new transformation array
  dx = transforms_smooth[i,0]
  dy = transforms_smooth[i,1]
  da = transforms_smooth[i,2]

  # Reconstruct transformation matrix accordingly to new values
  m = np.zeros((2,3), np.float32)
  m[0,0] = np.cos(da)
  m[0,1] = -np.sin(da)
  m[1,0] = np.sin(da)
  m[1,1] = np.cos(da)
  m[0,2] = dx
  m[1,2] = dy

  # Apply affine wrapping to the given frame
  frame_stabilized = cv2.warpAffine(frame, m, (w,h))

  # Fix border artifacts
  frame_stabilized = fixBorder(frame_stabilized)

  # Write the frame to the file
  frame_out = cv2.hconcat([frame, frame_stabilized])

  # If the image is too big, resize it.
  if(frame_out.shape[1] > w):
    frame_out = cv2.resize(frame_out, (frame_out.shape[1]//2, frame_out.shape[0]//2))

  cv2.imshow("Before and After", frame_out)
  cv2.waitKey(10)
  out.write(frame_stabilized)

cv2.destroyAllWindows()
out.release()

In [26]:
print(frame_stabilized.shape)

(720, 1280, 3)
